Project SNA by Aleksandra Elena Getman (r0884498) and Vaishnav Dilip (r0872689)

![Javatpoint](https://awoiaf.westeros.org/images/f/f6/Agot_hbo_guide_map.jpg) 

# Comunity mining in different Season

In the following section, we will apply the Leiden Algorithm to find communities in different seasons. Because various characters tend to betray each other and therefore alliances are often shifted, it is better to apply Leiden Algorithm as it can derive always a fixed number of communities. On top of that, it will be interesting to see how the connections between characters change between each other. 

After applying the algorithm, we will employ the 5 centrality measures to derive a bigger picture of the characters that are important. The 5 centrality measures will also assign the same meaning as in https://networkofthrones.wordpress.com/, as they carried out similar research.

Thus in where; 
1. Degree centrality: how many people the character knows?
2. Weighted degree centrality: how many interactions does the character have?
3. Eigenvector centrality: how many important people does the character know?
4. PageRank: how many important interactions does the character have?
5. Betweenness: does the character help to connect the whole network?

__Important note__

Recall that we plan to do link predictions for seasons 7 and 8 in the third notebook. This second notebook will serve as an explanatory analysis for the link prediction step. Looking ahead already, we would like to share that seasons 1 up until 4 will serve as a training set, seasons 5 and 6 will be used for the validation set, and seasons 7 and 8 will serve as the testing set. Therefore, community mining and centrality measures will be applied 3 times; 
1. For the training set
2. For the validation set
3. For testing set

In [50]:
#Connecting to Neo4j and loading the library

import pandas as pd
import numpy as np

from py2neo import Graph

graph = Graph("bolt://localhost:7687", auth=("neo4j", "neo4jneo4j"))

## Explanatory analysis for Training set (Seasons 1 up until 4)

### Leiden Algorithm

In [51]:
query = """
CALL gds.graph.drop('Got1') YIELD graphName;
"""

graph.run(query)


graphName
Got1


In [52]:
#Building a graph with seasons 1, 2, 3, and 4
query = """ 
CALL gds.graph.project(
    'Got1',
    {
        Person_1: {properties: 'seed' },
        Person_2: {properties: 'seed' },
        Person_3: {properties: 'seed' },
        Person_4: {properties: 'seed' }
    },
    {
        INTERACTS_1: {
            orientation: 'UNDIRECTED',
            properties: 'weight'
        },
        INTERACTS_2: {
            orientation: 'UNDIRECTED',
            properties: 'weight'
        },
        INTERACTS_3: {
            orientation: 'UNDIRECTED',
            properties: 'weight'
        },
        INTERACTS_4: {
            orientation: 'UNDIRECTED',
            properties: 'weight'
        }
    }
)
"""

graph.run(query)

nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,projectMillis
"{Person_1: {label: 'Person_1', properties: {seed: {defaultValue: null, property: 'seed'}}}, Person_2: {label: 'Person_2', properties: {seed: {defaultValue: null, property: 'seed'}}}, Person_3: {label: 'Person_3', properties: {seed: {defaultValue: null, property: 'seed'}}}, Person_4: {label: 'Person_4', properties: {seed: {defaultValue: null, property: 'seed'}}}}","{INTERACTS_1: {orientation: 'UNDIRECTED', indexInverse: false, aggregation: 'DEFAULT', type: 'INTERACTS_1', properties: {weight: {defaultValue: null, property: 'weight', aggregation: 'DEFAULT'}}}, INTERACTS_4: {orientation: 'UNDIRECTED', indexInverse: false, aggregation: 'DEFAULT', type: 'INTERACTS_4', properties: {weight: {defaultValue: null, property: 'weight', aggregation: 'DEFAULT'}}}, INTERACTS_2: {orientation: 'UNDIRECTED', indexInverse: false, aggregation: 'DEFAULT', type: 'INTERACTS_2', properties: {weight: {defaultValue: null, property: 'weight', aggregation: 'DEFAULT'}}}, INTERACTS_3: {orientation: 'UNDIRECTED', indexInverse: false, aggregation: 'DEFAULT', type: 'INTERACTS_3', properties: {weight: {defaultValue: null, property: 'weight', aggregation: 'DEFAULT'}}}}",Got1,317,4494,23


In [53]:
# The following will estimate the memory requirements for running the algorithm:
query = """
CALL gds.beta.leiden.write.estimate('Got1', { randomSeed:19, writeProperty: 'community_S1234' })
YIELD nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory
"""
graph.run(query)

nodeCount,relationshipCount,bytesMin,bytesMax,requiredMemory
317,4494,644504,653328,[629 KiB ... 638 KiB]


In [54]:
# The following will run the algorithm and stream results:
query = """
CALL gds.beta.leiden.stream('Got1', {randomSeed:19, relationshipWeightProperty: 'weight'})
YIELD nodeId, communityId, intermediateCommunityIds
RETURN gds.util.asNode(nodeId).label AS label, communityId
ORDER BY label ASC
"""
members = graph.run(query).to_data_frame()

In [55]:
members.groupby('communityId')['label'].apply(list)

communityId
0     [Balon Dwarf, Joffrey Dwarf, Renly Dwarf, Robb...
1     [Addam, Anya Waynwood, Boros, Bronn, Cersei, D...
2     [Aemon, Alliser Thorne, Allister, Benjen, Borc...
5     [Barristan, Cohollo, Daario, Daenerys, Doreah,...
6                             [Black Jack, Kegs, Mully]
8     [Alton, Assassin, Black Walder, Brienne, Brynd...
9     [Aerys, Amory, Anguy, Arya, Baelor, Barra, Ber...
11    [Adrack Humble, Baby, Balon, Billy, Black Lorr...
12    [Aegon, Axell Florent, Balerion, Bert, Cressen...
Name: label, dtype: object

In [16]:
members.groupby('communityId')['label'].apply(list)[0] # Community in with Drogo and the east

['Addam',
 'Adrack Humble',
 'Aegon',
 'Alton',
 'Anya Waynwood',
 'Assassin',
 'Axell Florent',
 'Baby',
 'Balerion',
 'Balon',
 'Balon Dwarf',
 'Bert',
 'Billy',
 'Black Lorren',
 'Black Walder',
 'Bolton Guard',
 'Boros',
 'Bran',
 'Brienne',
 'Bronn',
 'Brynden',
 'Captains Daughter',
 'Catelyn',
 'Cersei',
 'Colen',
 'Cressen',
 'Dagmer',
 'Daisy',
 'Davos',
 'Desmond',
 'Dontos',
 'Doran',
 'Dornish Lord',
 'Drowned Priest',
 'Edmure',
 'Eldrick',
 'Elia',
 'Ellaria',
 'Endrew',
 'Falyse',
 'Farlen',
 'Father Seaworth',
 'First Mate',
 'Fool',
 'Frey Daughter',
 'Frey Septon',
 'Galbart',
 'Gared',
 'Gerard',
 'Greatjon',
 'Gregor',
 'Harren',
 'Haylene',
 'High Septon',
 'Hodor',
 'Hoster',
 'Ilyn',
 'Jacks',
 'Jaime',
 'Joanna',
 'Joffrey',
 'Joffrey Dwarf',
 'Jojen',
 'Jonos',
 'Joyeuse',
 'Kevan',
 'Lancel',
 'Leaf',
 'Leo',
 'Lhara',
 'Locke',
 'Lollys',
 'Loras',
 'Lothar',
 'Luthor',
 'Luwin',
 'Lysa',
 'Mac',
 'Mace',
 'Mandon',
 'Marei',
 'Margaery',
 'Marillion',
 'Mart

In [17]:
members.groupby('communityId')['label'].apply(list)[1] # King's Landing

['Black Jack', 'Kegs', 'Mully']

In [18]:
members.groupby('communityId')['label'].apply(list)[5] # The Night's Watch

['Aemon',
 'Alliser Thorne',
 'Allister',
 'Benjen',
 'Borcas',
 'Bowen',
 'Cooper',
 'Craster',
 'Dareon',
 'Dongo the Giant',
 'Donnel Hill',
 'Eddison',
 'Farmer',
 'Gilly',
 'Grenn',
 'Guymon',
 'Hobb',
 'Janos',
 'Jaremy',
 'Jeor',
 'Jon',
 'Karl',
 'Little Sam',
 'Lord of Bones',
 'Luke',
 'Mag the Mighty',
 'Mance',
 'Melessa',
 'Moles Town Madam',
 'Moles Town Whore',
 'Morag',
 'Olly',
 'Ollys Mother',
 'Orell',
 'Othell',
 'Othor',
 'Pyp',
 'Qhorin',
 'Qhorin',
 'Quorin',
 'Randyll',
 'Rast',
 'Sam',
 'Sissy',
 'Styr',
 'Thenn Warg',
 'Tormund',
 'Ygritte']

In [19]:
members.groupby('communityId')['label'].apply(list)[11]


['Aerys',
 'Amory',
 'Anguy',
 'Arya',
 'Baelor',
 'Barra',
 'Beric',
 'Biter',
 'Brandon',
 'Donnel Waynwood',
 'Dying Man',
 'Gendry',
 'Hamlet Farmer',
 'Hot Pie',
 'Hugh of the Vale',
 'Innkeeper',
 'Innkeepers Daughter',
 'Jaqen',
 'Jon Arryn',
 'Jory',
 'Lommy',
 'Lommy',
 'Lyanna',
 'Mhaegen',
 'Ned',
 'Polliver',
 'Rhaegar',
 'Rhaenys',
 'Rickard Stark',
 'Robert',
 'Rorge',
 'Sally',
 'Sandor',
 'Stable Boy',
 'Steffon',
 'Syrio',
 'Syrio Forel',
 'Ternesio Terys',
 'Thoros',
 'Tickler',
 'Tobho Mott',
 'Tomard',
 'Tommy',
 'Varly',
 'Visenya',
 'Yoren']

In [20]:
members.groupby('communityId')['label'].apply(list)[14] # The North and further north

KeyError: 14

In [21]:
# The following will run the algorithm and returns the result in form of statistical and measurement values:
query = """
CALL gds.beta.leiden.stats('Got1', {randomSeed:19 })
YIELD communityCount
"""
graph.run(query)

communityCount
8


In [8]:
# The following run the algorithm, and write back results:
query = """
CALL gds.beta.leiden.write('Got1', { randomSeed:19, writeProperty: 'community_S1234' })
YIELD communityCount, modularity, modularities
"""
graph.run(query)

communityCount,modularity,modularities
8,0.4929617871546674,"[0.47254461221994254, 0.48356667615367693, 0.4929617871546674]"


<img src="images\bloom-visualisation_season1234.png"
     style="float: left; margin-right: 10px;" />

### 5 Centrality Measures

In [9]:
# Degree
query = """
CALL gds.degree.stream('Got1')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query).to_data_frame()

,name,score
0,TYRION,138.0
1,JOFFREY,125.0
2,CERSEI,114.0
3,NED,110.0
4,TYWIN,100.0


In [10]:
# Weighted Degree
query = """
CALL gds.degree.stream(
   'Got1',
   { relationshipWeightProperty: 'weight' }
)
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query).to_data_frame()

,name,score
0,TYRION,3212.0
1,CERSEI,1979.0
2,JON,1791.0
3,SANSA,1585.0
4,DAENERYS,1561.0


In [11]:
# Eigenvector
query = """
CALL gds.eigenvector.stream('Got1')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query).to_data_frame()

,name,score
0,JOFFREY,0.296271
1,TYRION,0.282599
2,CERSEI,0.275806
3,SANSA,0.247159
4,TYWIN,0.242574


In [12]:
# Pagerank
query = """
CALL gds.pageRank.stream('Got1',  {maxIterations: 20,  dampingFactor: 0.85})
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS page, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query).to_data_frame()

,page,score
0,TYRION,7.631802
1,JOFFREY,6.333186
2,NED,6.022508
3,ARYA,6.019449
4,CERSEI,5.648543


In [13]:
# Betweenness
query = """
CALL gds.betweenness.stream('Got1')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query).to_data_frame()

,name,score
0,TYRION,6869.916676
1,DAENERYS,6867.406307
2,NED,6765.354456
3,JON,5999.762027
4,ARYA,5068.881863


### Discussion

<tr>
    <td><img src="https://upload.wikimedia.org/wikipedia/en/e/e8/Game_of_Thrones_Season_1.jpg" width=270 height=480></td>
    <td><img src="https://upload.wikimedia.org/wikipedia/en/c/c2/Game_of_Thrones_Season_2.jpg" width=270 height=480></td>
    <td><img src="https://upload.wikimedia.org/wikipedia/en/1/1d/Game_of_Thrones_Season_3.jpg" width=270 height=480></td>
    <td><img src="https://upload.wikimedia.org/wikipedia/en/8/80/Game_of_Thrones_Season_4.jpg" width=270 height=480></td>
</tr>

Analyzing the overall series of GOT starting from one up until 4 is a bit tricky as a lot has happened. Yet whenever looking at the bigger picture of all the centrality measures four names stand out: Tyrion, Joffrey, Cersei, and Ned.

Ned has a high degree, PageRank, and betweenness. In season 1, Ned played the leading role as he was offered to be the right hand of Robert (king of the Iron Throne in season 1). Due to his position, he knew a lot of people and interacted also with a lot of important officials (this explains elevated levels of degree and PageRank). On top of this, Ned found out that Joffrey (crown prince) was not the biological son of Robert and that Cersei (the queen of the Iron Throne in season 1 and mother of Joffrey) cheated on the king with her older brother Jaime. Ned wanted to share this valuable information with Robert, yet he was unable to do so as Robert died in an accident. Before Ned had a chance to shame Cersei and take away the title from Joffrey, he was executed for so elledge treason (given by the command of Cersei). Ned was a well-respected man and was from the house of Starks. His death meant war and revenge. The fact that Ned knew the insest and the hunger for power Cersei, led him to become the center node of the graph, as he connected all the dots (hence this is the explanation for his high betweenness).

In the series, Cersei plays a noticeably crucial role in making different arrangements in order to ensure that the Iron Throne stays inside the family, and she is willing to go as far as killing other people to ensure the power. All of this explains the high levels of degree, weighted degree, eigenvector, and PageRank.

After the death of Robert and Ned, Joffrey became the next king of the Iron Throne. This explains his high levels of degree and eigenvector (i.e., knowing a lot of people (also influential politicians as Joffrey is now involved in the ruling of the kingdom) and interacting also with a lot of influential people (hence high levels of PageRank). During his rule, he assigns his grandfather Tywin (also father to Cersei) to be his right hand. But due to the war, Tywin gives his position to his younger son, Tyrion.

During the whole show of The Game of Thrones, Tyrion feels left out by his family as he is being neglected and blamed by Cersei, Tywin, and Jamie for the death of his mother who died giving birth to him. Even when Tyrion becomes the right hand of Joffrey, he is not treated with respect or taken seriously, instead, people make fun of him. But Tyrion reaches his boiling point when he is accused of killing Joffrey with poison by Cersei, this allegation let him to change sides and flee to the other side of the kingdom where he meets Daenerys (i.e., the last descendant of Targaryen, her family initially owned the Iron Throne). In later seasons he will prove to be useful to her as he knows a lot about King's Landing (where the Iron Throne is situated). She is on a mission to reclaim her birthright to the throne and Tyrion is going to help her do this as revenge on Cersei. Notice how Tyrion is on the top of four out of five centrality measures. This means that he knows a lot of people (also important people), interacts a lot with important characters, and helps the network to connect overall (i.e., high degree, weighted degree, PageRank, and betweenness).

## Explanatory analysis for Validation set (Seasons 5 and 6)

### Leiden Algorithm

In [14]:
#Building a grpah with season 5 and 6
query = """ 
CALL gds.graph.project(
    'Got2',
    {
        Person_5: {properties: 'seed' },
        Person_6: {properties: 'seed' }
    },
    {
        INTERACTS_5: {
            orientation: 'UNDIRECTED',
            properties: 'weight'
        },
        INTERACTS_6: {
            orientation: 'UNDIRECTED',
            properties: 'weight'
        }
    }
)
"""

graph.run(query)

nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,projectMillis
"{Person_6: {label: 'Person_6', properties: {seed: {defaultValue: null, property: 'seed'}}}, Person_5: {label: 'Person_5', properties: {seed: {defaultValue: null, property: 'seed'}}}}","{INTERACTS_6: {orientation: 'UNDIRECTED', indexInverse: false, aggregation: 'DEFAULT', type: 'INTERACTS_6', properties: {weight: {defaultValue: null, property: 'weight', aggregation: 'DEFAULT'}}}, INTERACTS_5: {orientation: 'UNDIRECTED', indexInverse: false, aggregation: 'DEFAULT', type: 'INTERACTS_5', properties: {weight: {defaultValue: null, property: 'weight', aggregation: 'DEFAULT'}}}}",Got2,188,1892,32


In [15]:
# The following will estimate the memory requirements for running the algorithm:
query = """
CALL gds.beta.leiden.write.estimate('Got2', { writeProperty: 'community_S56' })
YIELD nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory
"""
graph.run(query)

nodeCount,relationshipCount,bytesMin,bytesMax,requiredMemory
188,1892,610416,615624,[596 KiB ... 601 KiB]


In [16]:
# The following will run the algorithm and stream results:
query = """
CALL gds.beta.leiden.stream('Got2', {seedProperty: 'seed', relationshipWeightProperty: 'weight'})
YIELD nodeId, communityId, intermediateCommunityIds
RETURN gds.util.asNode(nodeId).label AS label, communityId
ORDER BY label ASC
"""
graph.run(query).to_data_frame()

,label,communityId
0,Aegon,0
1,Aemon,0
2,Aeron,32
3,Aerys,10
4,Alliser,0
...,...,...
183,Wolkan,32
184,Wun Wun,0
185,Yara,32
186,Yezzan,10


In [17]:
# The following will run the algorithm and returns the result in form of statistical and measurement values:
query = """
CALL gds.beta.leiden.stats('Got2')
YIELD communityCount
"""
graph.run(query)

communityCount
8


In [18]:
# The following run the algorithm, and write back results:
query = """
CALL gds.beta.leiden.write('Got2', { writeProperty: 'community_S56' })
YIELD communityCount, modularity, modularities
"""
graph.run(query)

communityCount,modularity,modularities
8,0.6052791546916135,"[0.5687645544386289, 0.594723974093658, 0.6052791546916135]"


<img src="images\bloom-visualisation_season56.png"
     style="float: left; margin-right: 10px;" />

### 5 Centrality Measures

In [19]:
# Degree
query = """
CALL gds.degree.stream('Got2')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query).to_data_frame()

,name,score
0,SANSA,62.0
1,CERSEI,58.0
2,JON,56.0
3,TYRION,45.0
4,JAIME,41.0


In [20]:
# Weighted Degree
query = """
CALL gds.degree.stream(
   'Got2',
   { relationshipWeightProperty: 'weight' }
)
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query).to_data_frame()

,name,score
0,JON,1150.0
1,CERSEI,924.0
2,TYRION,922.0
3,SANSA,913.0
4,JAIME,735.0


In [21]:
# Eigenvector
query = """
CALL gds.eigenvector.stream('Got2')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query).to_data_frame()

,name,score
0,SANSA,0.326439
1,CERSEI,0.270071
2,JON,0.250806
3,LITTLEFINGER,0.201531
4,STANNIS,0.199998


In [22]:
# Pagerank
query = """
CALL gds.pageRank.stream('Got2',  {maxIterations: 20,  dampingFactor: 0.85})
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS page, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query).to_data_frame()

,page,score
0,JON,4.703863
1,SANSA,4.685592
2,CERSEI,4.273345
3,TYRION,3.870091
4,JAIME,3.251898


In [23]:
# Betweenness
query = """
CALL gds.betweenness.stream('Got2')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query).to_data_frame()

,name,score
0,SANSA,3249.913507
1,JON,2809.363537
2,TYRION,2453.560830
3,JAIME,2244.448242
4,CERSEI,2073.401141


### Discussion

<tr>
    <td><img src="https://upload.wikimedia.org/wikipedia/en/5/59/Game_of_Thrones_Season_5.png" width=270 height=480></td>
    <td><img src="https://upload.wikimedia.org/wikipedia/en/d/d1/Game_of_Thrones_Season_6.jpeg" width=270 height=480></td>
</tr>

Season 5 and 6 are two of the most important seasons in the series. These are the seasons where the characters move to their allies and major power struggles begin. Sansa, Jon and Cersie are seen to be the most important people in these seasons. They top all the 5 centrality categories. Several important events involving these characters are seen in these seasons. Jon gets killed and rises back from the dead, Sansa wins the battle of Bastards for Jon and Cersei demolishes all her enemies in the Kings Landing. These are the seasons where the show picks up pace and is on a cruise to the inevitable War against the White Walkers. 

## Explanatory analysis for Testing set (Seasons 7 and 8)

### Leiden Algorithm

In [12]:
#Building a graph with season 7 and 8
query = """ 
CALL gds.graph.project(
    'Got3',
    {
        Person_7: {properties: 'seed' },
        Person_8: {properties: 'seed' }
    },
    {
        INTERACTS_7: {
            orientation: 'UNDIRECTED',
            properties: 'weight'
        },
        INTERACTS_8: {
            orientation: 'UNDIRECTED',
            properties: 'weight'
        }
    }
)
"""

graph.run(query)

nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,projectMillis
"{Person_7: {label: 'Person_7', properties: {seed: {defaultValue: null, property: 'seed'}}}, Person_8: {label: 'Person_8', properties: {seed: {defaultValue: null, property: 'seed'}}}}","{INTERACTS_8: {orientation: 'UNDIRECTED', indexInverse: false, aggregation: 'DEFAULT', type: 'INTERACTS_8', properties: {weight: {defaultValue: null, property: 'weight', aggregation: 'DEFAULT'}}}, INTERACTS_7: {orientation: 'UNDIRECTED', indexInverse: false, aggregation: 'DEFAULT', type: 'INTERACTS_7', properties: {weight: {defaultValue: null, property: 'weight', aggregation: 'DEFAULT'}}}}",Got3,104,1932,28


In [13]:
# The following will run the algorithm and stream results:
query = """
CALL gds.beta.leiden.stream('Got3', {seedProperty: 'seed', relationshipWeightProperty: 'weight'})
YIELD nodeId, communityId, intermediateCommunityIds
RETURN gds.util.asNode(nodeId).label AS label, communityId
ORDER BY label ASC
"""
members = graph.run(query).to_data_frame()

In [15]:
members.groupby('communityId')['label'].apply(list)

communityId
0    [Beric, Eddison, Gendry, Gilly, High Septon (A...
1    [Aegon, Aerys, Benjen, Daenerys, Davos, Drogo,...
6    [Alanna, Alys, Arya, Bran, Catelyn, Dornish Pr...
7    [Alton, Balerion, Brienne, Bronn, Cersei, Cers...
Name: label, dtype: object

In [26]:
# The following will run the algorithm and returns the result in form of statistical and measurement values:
query = """
CALL gds.beta.leiden.stats('Got3')
YIELD communityCount
"""
graph.run(query)

communityCount
4


In [27]:
# The following will run the algorithm and store the results in myGraph:
query = """
CALL gds.beta.leiden.mutate('Got3', { mutateProperty: 'communityId' })
YIELD communityCount, modularity, modularities
"""
graph.run(query)

communityCount,modularity,modularities
5,0.22509151738830377,"[0.20916170500966613, 0.22406489375838554, 0.22509151738830377]"


In [28]:
# The following run the algorithm, and write back results:
query = """
CALL gds.beta.leiden.write('Got3', { writeProperty: 'community_S78' })
YIELD communityCount, modularity, modularities
"""
graph.run(query)

communityCount,modularity,modularities
5,0.22708207416552004,"[0.2122415758994209, 0.22456802506761997, 0.22708207416552004]"


<img src="images\bloom-visualisation_season78.png"
     style="float: left; margin-right: 10px;" />

### 5 Centrality Measures

In [29]:
# Degree
query = """
CALL gds.degree.stream('Got3')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query).to_data_frame()

,name,score
0,JON,79.0
1,TYRION,75.0
2,DAENERYS,75.0
3,SANSA,64.0
4,DAVOS,64.0


In [30]:
# Weighted Degree
query = """
CALL gds.degree.stream(
   'Got3',
   { relationshipWeightProperty: 'weight' }
)
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query).to_data_frame()

,name,score
0,JON,1882.0
1,TYRION,1755.0
2,DAENERYS,1576.0
3,SANSA,1119.0
4,JAIME,1107.0


In [31]:
# Eigenvector
query = """
CALL gds.eigenvector.stream('Got3')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query).to_data_frame()

,name,score
0,JON,0.254100
1,TYRION,0.244820
2,DAENERYS,0.235475
3,DAVOS,0.233078
4,BRIENNE,0.216607


In [32]:
# Pagerank
query = """
CALL gds.pageRank.stream('Got3',  {maxIterations: 20,  dampingFactor: 0.85})
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS page, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query).to_data_frame()

,page,score
0,JON,3.667777
1,DAENERYS,3.509703
2,TYRION,3.299509
3,SANSA,3.073163
4,ARYA,2.865504


In [33]:
# Betweenness
query = """
CALL gds.betweenness.stream('Got3')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query).to_data_frame()

,name,score
0,JON,542.696894
1,ARYA,513.175492
2,SAM,512.808655
3,SANSA,497.389139
4,DAENERYS,471.243228


### Discussion

<tr>
    <td><img src="https://upload.wikimedia.org/wikipedia/en/9/92/Game_of_Thrones_Season_7.png" width=270 height=480></td>
    <td><img src="https://upload.wikimedia.org/wikipedia/en/e/e0/Game_of_Thrones_Season_8.png" width=270 height=480></td>
</tr>

Season 7 and 8, the final seasons of the series is full of twists and turns. We see the North and South team up to defeat the White Walkers, the battle for Kings Landing and finally Jon Snow killing Daerneys. The least expected Bran becomes the King and Tyrion his Hand. The North remains an independent Domain. The Centrality metrics paint the clear picture of Jon being the most important character in the last two seasons. He was indispensible in uniting the North and the South to fight against the White Walkers. The Starks are on a whole seen to rise up in the centrality ranks ascertaining the fact that Game of thrones is indeed the story of the Starks. Daerneys also rises in the ranks owing to her contribution to the storyline.

Based on this explanatory analysis, let's move to the third notebook which is about link prediction.